
# Assignment 5: Simulating Call Option Prices using GBM

In this notebook, we simulate and estimate the prices of different types of call options using the Euler–Maruyama method and Monte Carlo simulation.

The underlying asset follows a Geometric Brownian Motion (GBM), and the options priced are:

- European Call Option
- Arithmetic Asian Call Option
- Geometric Asian Call Option
- Lookback Call Option (floating strike)

Assumptions:
- The asset price follows GBM: dS_t = mu * S_t * dt + sigma * S_t * dW_t
- We are working under the risk-neutral measure, so mu = r
- No dividends or transaction costs
- Euler–Maruyama discretization is used for simulation


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
S0 = 100      # Initial asset price
K = 100       # Strike price
T = 1.0       # Time to maturity (in years)
r = 0.05      # Risk-free interest rate
sigma = 0.2   # Volatility
M = 100       # Number of time steps
N = 10000     # Number of simulation paths
dt = T / M

print(f"Simulating {N} paths over {M} time steps...")


In [ ]:

def simulate_gbm_paths(S0, r, sigma, T, M, N):
    dt = T / M
    paths = np.zeros((N, M + 1))
    paths[:, 0] = S0
    for t in range(1, M + 1):
        Z = np.random.normal(0, 1, N)
        paths[:, t] = paths[:, t - 1] * (1 + r * dt + sigma * np.sqrt(dt) * Z)
    return paths


In [ ]:

def european_call_price(paths, K, r, T):
    payoff = np.maximum(paths[:, -1] - K, 0)
    return np.exp(-r * T) * np.mean(payoff)

def arithmetic_asian_call_price(paths, K, r, T):
    average_price = np.mean(paths[:, 1:], axis=1)
    payoff = np.maximum(average_price - K, 0)
    return np.exp(-r * T) * np.mean(payoff)

def geometric_asian_call_price(paths, K, r, T):
    log_paths = np.log(paths[:, 1:])
    geo_average = np.exp(np.mean(log_paths, axis=1))
    payoff = np.maximum(geo_average - K, 0)
    return np.exp(-r * T) * np.mean(payoff)

def lookback_call_floating_strike_price(paths, r, T):
    min_price = np.min(paths[:, 1:], axis=1)
    payoff = np.maximum(paths[:, -1] - min_price, 0)
    return np.exp(-r * T) * np.mean(payoff)


In [ ]:

paths = simulate_gbm_paths(S0, r, sigma, T, M, N)

print("European Call Option Price:", round(european_call_price(paths, K, r, T), 4))
print("Arithmetic Asian Call Option Price:", round(arithmetic_asian_call_price(paths, K, r, T), 4))
print("Geometric Asian Call Option Price:", round(geometric_asian_call_price(paths, K, r, T), 4))
print("Lookback (Floating Strike) Call Option Price:", round(lookback_call_floating_strike_price(paths, r, T), 4))


In [ ]:

plt.figure(figsize=(10, 6))
for i in range(10):
    plt.plot(np.linspace(0, T, M + 1), paths[i], lw=1)
plt.title('Sample GBM Asset Price Paths')
plt.xlabel('Time (Years)')
plt.ylabel('Asset Price')
plt.grid(True)
plt.show()



Summary:

- Asset paths were simulated using the Euler–Maruyama method.
- Prices of four types of call options were estimated using Monte Carlo simulation.
- Each option type required a different payoff function applied at maturity.
